# EuroPython program grid

In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 99999;
//increase max size of output area

<IPython.core.display.Javascript object>

In [3]:
import json
import datetime as dt
from random import choice, randrange, shuffle
from copy import deepcopy

from functools import partial
from operator import itemgetter

from IPython.display import display, HTML
show = lambda s: display(HTML(s))

In [121]:
talk_sessions = json.load(open('accepted_talks.json'))

In [122]:
list(talk_sessions.keys())

['training',
 'poster',
 'EPS session',
 'Opening session',
 'Lightning talk',
 'Closing session',
 'talk']

In [123]:
#all talks
all_talks = []
for s in talk_sessions.values():
    all_talks.extend(list(s.values()))

#the talks worth for scheduling
grid_talks = []
sessions = talk_sessions.copy()
general_grid_sessions = ['talk', 'EPS session', 'training']
for session_name in general_grid_sessions:
    grid_talks.extend(sessions[session_name].values())

fields2pop = ['abstract_extra', 
              'abstract_long',
              'abstract_short',
              'twitters',
              'emails',
              'status',
              'url',
              'companies',
              'adm_type',
              'tag_categories',
              'have_tickets',
             ]
for talk in grid_talks:
    for f in fields2pop:
        talk.pop(f)

In [132]:
import datetime
from   collections import OrderedDict
from   functools   import partial

# talk rooms
# ((h, m), duration)
type1_schedule = [((11,  0), 45), 
                  ((11, 45), 45),
                  ((12, 30), 45),
                  ((14, 30), 45),
                  ((15, 15), 30),
                  ((15, 45), 30),
                  ((17,  0), 30)]

type2_schedule = [((11,  0), 45), 
                  ((11, 45), 45),
                  ((12, 30), 45),
                  ((14, 30), 45),
                  ((15, 15), 60),
                  ((17,  0), 30)]

type3_schedule = [((11,  0), 45), 
                  ((11, 45), 45),
                  ((12, 30), 30),
                  ((14, 30), 60),
                  ((15, 15), 30),
                  ((15, 45), 30),
                  ((17,  0), 30)]

#tutorials
type4_schedule = [((11,  0), 180),
                  ((13, 30), 180)]

keynote_time   = (( 9, 30), 60)
lightning_time = ((17, 30), 60) 
lunch_time     = ((13,  0), 90) 
am_coffee_time = ((10, 30), 30)
pm_coffee_time = ((16, 15), 45) 

breaks = [(keynote_time,   'Keynote speech'),
          (lunch_time,     'Lunch'),
          (am_coffee_time, 'Coffee break'),
          (pm_coffee_time, 'Coffee break'),
          (lightning_time, 'Lightning talks')]

daily_timegrid = lambda schedule: OrderedDict([(datetime.time(*slot[0]), slot[1]) for slot in schedule])

room1_schedule = daily_timegrid(type1_schedule) # A1, the google room
room2_schedule = daily_timegrid(type2_schedule) # A3, pythonanywhere room
room3_schedule = daily_timegrid(type3_schedule) # A2
room4_schedule = daily_timegrid(type3_schedule) # Barria1
room5_schedule = daily_timegrid(type3_schedule) # Barria2

room6_schedule = daily_timegrid(type4_schedule) # Room E
room7_schedule = daily_timegrid(type4_schedule) # Room A4

daily_schedule = OrderedDict([('A1' , room1_schedule), 
                              ('A3' , room2_schedule),
                              ('A2' , room3_schedule),
                              ('Ba1', room4_schedule),
                              ('Ba2', room5_schedule),
                              ('E'  , room6_schedule),
                              ('A4' , room7_schedule)])

# slots_durations
slots_durations = [list(slots.values()) for slots in list(daily_schedule.values())]
slots = [val for sublist in slots_durations for val in sublist]

In [133]:
def find_talk(talk_title):
    talks = []
    for talk in all_talks:
        if talk_title in talk['title']:
            talks.append(talk)
    return talks

find_talk('Python and PyPy performance')

[]

In [134]:
tags = {}
text = []
for talk in all_talks:
    for t in talk['tags']:
        if t.lower() == 'python':
            continue
        if not t.lower() in tags:
            tags[t.lower()] = 0
        tags[t.lower()] += 1
        text.append(t.lower())
tags_sorted = sorted(tags.items(), key=itemgetter(1), reverse=True)

In [135]:
def pick_talk(talks, tags, duration):
    if not talks:
        raise IndexError('The list of talks is empty!')

    while tags:
        randidx = randrange(0, len(tags))
        atag    = tags.pop(randidx)[0]
        for tlkidx, talk in enumerate(talks):
            talk_tags = [t.lower() for t in talk['tags']]
            if talk['duration'] == duration and atag in talk_tags:
                atalk = talks.pop(tlkidx)
                return atalk

    return pick_talk_by_duration(talks, duration)


def pick_talk_by_duration(talks, duration):
    if not talks:
        raise IndexError('The list of talks is empty!')

    for tlkidx, talk in enumerate(talks):
        if talk['duration'] == duration:
                atalk = talks.pop(tlkidx)
                return atalk
    return {}

In [136]:
talks = grid_talks.copy()
shuffle(talks)

# random pick talks
week_talks = []
n_days = 5
for d in range(n_days):
    day_talks = []
    for slot in slots:
        day_tags = tags_sorted.copy()
        try:
            day_talks.append(pick_talk(talks, day_tags, slot))
        except:
            print('No talks left.')
                    
    week_talks.append(day_talks)

In [137]:
weekly_schedule = OrderedDict()

for d in range(n_days):
    weekly_schedule[d] = deepcopy(daily_schedule)
    day_talks          = week_talks[d].copy()
    for room in daily_schedule:
        for slot in daily_schedule[room]:
            #print(weekly_schedule[d][room].keys())
            weekly_schedule[d][room][slot] = day_talks.pop(0)
            #print(day_talks)

In [156]:
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)


def make_daily_schedule(time_list, header=''):
    table = ListTable()
    table.append(header)
    for slot in time_list:
        table.append([slot] + time_list[slot])
    return table


def get_room_schedule(weekly_schedule, room_name):
    slots = list(daily_schedule[room_name].keys())
    daily_slots = []
    for slot in slots:
        talks = [weekly_schedule[d][room_name][slot].get('title', '-') for d in range(n_days)]
        daily_slots.append((slot, talks))
    room_schedule = OrderedDict(daily_slots)
    return room_schedule


weekday_names = ['Monday, July 18th', 
                 'Tuesday, July 19th',
                 'Wednesday, July 20th',
                 'Thursday, July 21st',
                 'Friday, July 22nd']

In [157]:
show('<h1>Not scheduled talks</h1>')

for talk in talks:
    print(talk)

# TALKS

### The Google Room

In [158]:
make_daily_schedule(get_room_schedule(weekly_schedule, 'A1'), header=['Google Room'] + weekday_names)

Google Room,"Monday, July 18th","Tuesday, July 19th","Wednesday, July 20th","Thursday, July 21st","Friday, July 22nd"
11:00:00,Introduction to aiohttp,Moving away from NodeJS to a pure python solution for assets,What Python can learn from Haskell packaging,Efficient Django,-
11:45:00,Against the silos: usable encrypted email & the quest for privacy-aware services,Using Service Discovery to build dynamic python applications,Python in Astronomy,Asynchronous network requests in a web application,-
12:30:00,What is the best full text search engine for Python?,Profiling the unprofilable,It's not magic: descriptors exposed,Brainwaves for hackers 3,-
14:30:00,Hacking ético con herramientas Python,Scaling Microservices with Crossbar.io,SQLAlchemy as the backbone of a Data Science company,A Gentle Introduction to Neural Networks (with Python),-
15:15:00,Using and abusing Python’s double-underscore methods and attributes,Writing Redis in Python with asyncio,Learn Python The Fun Way,Get in control of your workflows with Airflow,Protect your users with Circuit Breakers
15:45:00,Real virtual environments without virtualenv,"Server for IoT devices and Mobile devices using Wifi Network,",AFP: secure cloud authentication for machines and humans.,Building a mBaaS framework using Django,Implementing a Sound Identifier in Python
17:00:00,Go for Python Programmers,Machine Learning: Power of Ensembles,NetworkX Visualization Powered by Bokeh,Making robots walk with Python,Testing the untestable: a beginner’s guide to mock objects


### Room A2

In [159]:
make_daily_schedule(get_room_schedule(weekly_schedule, 'A2'), header=['A2'] + weekday_names)

A2,"Monday, July 18th","Tuesday, July 19th","Wednesday, July 20th","Thursday, July 21st","Friday, July 22nd"
11:00:00,CloudABI: Capability based security on Linux/UNIX,Do I need to switch to Go(lang) ?,Analyzing Data with Python & Docker,Designing a Pythonic Interface,-
11:45:00,Things I wish I knew before starting using Python for Data Processing,TDD of Python microservices,Towards More Secure Emailing,The Report Of Twisted’s Death,-
12:30:00,"Endor, ipuinak kontatzen zituen Nao robota.",EITB Nahieran: askatu bideoak API honen bidez,Machine Learning for dummies with Python,Writing Python Native Extensions in Rust,Wrestling Python into LLVM Intermediate Representation
14:30:00,APIs and Microservices With Go,Modern OpenGL with Python,-,-,-
15:15:00,Python as the keystone of building and testing C++ applications,Hands-on with nilearn for Neuroimaging,Music transcription with Python,AWS lambda & Python,Optimize Thyself
15:45:00,Behind Closed Doors: Managing Passwords in a Dangerous World,"Iteration, iteration, iteration",Is that spam in my ham?,Predicting Oscar Winners & Box Office Hits with Scikit Learn,Exploring our Python Interpreter
17:00:00,The Journey from Python Developer to Python Company Owner,Writing faster Python,Managing Mocks,The right way to write documentation,"Grocker, a Python build chain for Docker"


### The PythonAnywhere Room

In [160]:
make_daily_schedule(get_room_schedule(weekly_schedule, 'A3'), header=['PythonAnywhere Room'] + weekday_names)

PythonAnywhere Room,"Monday, July 18th","Tuesday, July 19th","Wednesday, July 20th","Thursday, July 21st","Friday, July 22nd"
11:00:00,Un vector por tu palabra,Kung Fu at Dawn with Itertools,Ethical hacking with Python tools,Handling GPS Data with Python,-
11:45:00,High Performance Networking in Python,Effective Code Review,Operating on Encrypted Data with ZeroDB,Python in Gravitational Waves Research Communities,-
12:30:00,Get Instrumented!,"Infrastructure as Code: ""pip install"" your environment",What's the point of Object Orientation?,Performant Python,-
14:30:00,Jupyter for everything else,How to improve your diet and save money with Python,Metaclasses for fun and profit: Making a declarative GUI implementation,Implementing Parallel Programming Design Patterns using EFL for Python,-
15:15:00,An Introduction to Deep Learning,PSF Meeting,-,-,-
17:00:00,Test Driven Deployment with Ansible 2.0,Ingesting 35 million hotel images with python in the cloud.,async/await in Python 3.5 and why it is awesome,Pygame Zero,RESTful API - Best Practises.


### Room Barria1

In [161]:
make_daily_schedule(get_room_schedule(weekly_schedule, 'Ba1'), header=['Barria1 Room'] + weekday_names)

Barria1 Room,"Monday, July 18th","Tuesday, July 19th","Wednesday, July 20th","Thursday, July 21st","Friday, July 22nd"
11:00:00,"OMG, Bokeh is better than ever!",The Joy of Simulation: for Fun and Profit,"I Hate You, NLP... ;)",Kung Fu al amanecer con itertools,-
11:45:00,"Beyond scraping, getting data from dynamic, heavily javascript driven, websites",How OpenStack makes Python better (and vice-versa),"Automate, contribute, repeat.",Fast Async Code with Cython and AsyncIO,-
12:30:00,"Python, Data & Rock'n'Roll",Simplifying Computer Art in Python,So you think your Python startup is worth $10 million...,Building a reasonably popular web application for the first time.,Python Descriptors for Better Data Structures
14:30:00,EPS General Assembly,Nipy on functional brain MRI,-,-,-
15:15:00,System Testing with pytest and docker-py,Building beautiful RESTful APIs using Flask,import community,CFFI: calling C from Python,Writing unit tests for C code in Python
15:45:00,Create secure production environment using Docker,Datu bistaratze soluzioen garapena Smartcity proiektuetan,Buildout Django eta Fabric. Kasu praktikoa euskarazko tokiko hedabideetan,MiniBrew: Brewing beer with Python,Exploring Python Bytecode
17:00:00,Implementación de un Identificador de Sonido en Python,Entendiendo Unicode,Dynamic Class Generation in Python,Keeping the Lights on with Python,The value of mindfulness and how it has arrived at Google


### Room Barria2

In [162]:
make_daily_schedule(get_room_schedule(weekly_schedule, 'Ba2'), header=['Barria2 Room'] + weekday_names)

Barria2 Room,"Monday, July 18th","Tuesday, July 19th","Wednesday, July 20th","Thursday, July 21st","Friday, July 22nd"
11:00:00,FAT Python: a new static optimizer for Python 3.6,Raspberry Pi GPIO Zero,Building Service interfaces with OpenAPI / Swagger,Build your first OpenStack application with OpenStack PythonSDK,-
11:45:00,Building your own AI,Data Formats for Data Science,Deep Learning with Python & TensorFlow,-,-
12:30:00,Conda - Easier Installs and Simpler Builds,Python and Async programming,Pure Data and a Clean Architecture,Game Theory to the Rescue When Hard Decisions Are to Be Made,Sponsored Talk
14:30:00,A deep dive into the Pymongo MongoDB driver,-,-,-,-
15:15:00,MicroPython on the BBC micro:bit,Peeking into Python’s C API,Developing a real-time automated trading platform with Python,re-Discovering Python's Regular Expressions,Sponsored Talk
15:45:00,Clean code in Python,How to use Metaclasses to improve your Software Design,Pytest desde las trincheras,Another pair of eyes: Reviewing code well,-
17:00:00,Monkey-patching: a magic trick or a powerful tool?,Effectively test your webapp with Python and Selenium,Build your Microservices with ZeroMQ,EuroPython 2017: Help us build the next edition!,-


# TRAININGS

### Room E

In [163]:
make_daily_schedule(get_room_schedule(weekly_schedule, 'E'), header=['E Room'] + weekday_names)

E Room,"Monday, July 18th","Tuesday, July 19th","Wednesday, July 20th","Thursday, July 21st","Friday, July 22nd"
11:00:00,uWSGI: the full stack application server,Present-day Async Web development training: from Twisted to Tornado and AsyncIO,OpenStack Cloud Native Deployment for Application Developers,"Faster Python Programs - Measure, don't Guess",Productive Coding with PyCharm
13:30:00,Python for System Administrators,Guide to make a real contribution to an open source project for novice,NumPy with Cython,Test-driven code search and the art of writing implementation-agnostic tests,Manage your Python packages professionally with devpi


### Room A4

In [164]:
make_daily_schedule(get_room_schedule(weekly_schedule, 'A4'), header=['A4 Room'] + weekday_names)

A4 Room,"Monday, July 18th","Tuesday, July 19th","Wednesday, July 20th","Thursday, July 21st","Friday, July 22nd"
11:00:00,The Stupid Python Workshop,Efficient Python for High-Performance Parallel Computing,"(Machine-)Learning Chinese, with Python!","Blender: much visual, very 3d, many python.",Property-based testing with Hypothesis
13:30:00,Introduction to Deep Learning for Natural Language Processing,"So, what's all the fuss about Docker?",Introduction to Data Wrangling,"pytest - simple, rapid and fun testing with Python",IPython in Depth
